# N-Gram

In [1]:
"""
단어들을 n개씩 묶는 것에 따라서 n-gram은 여러가지 이름을 가진다.
단순히 단어 한 두개로 문맥을 파악하기 어렵기 때문에, n-gram을 사용한다. 
1개 > unigram
2개 > bigram
3개 > trigram
4개 > fourgram
5개 > fivegram

trade-off
n을 너무 크게 설정하면, n-gram이 여러번 나타나지 않으면서 Out of Vocabulary(사전에 없음) 문제가 발생
n을 너무 작게 설정하면, count는 잘 되지만 정확도가 떨어질 수 있다. 

또한, 적용분야(Domain)에 맞는 코퍼스를 수집해야한다.
분야에 따라 단어들의 확률 분포가 다르기 때문에, 분야에 적합한 코퍼스를 사용하면 언어모델의 성능이 높아질 수 있다. 
"""
True

True

# Bayes Classifier
* 모델 중심이 아닌 데이터 중심: 조건부 확률에 기반
* 범주형 예측자료에만 적용 가능: 수치형 자료는 범주형으로 변환이 필요
* 대용량 데이터(학습시킬 데이터가 많을 수록 좋다.)  
**종류가 두 가지로 나뉜다.**
1. Exact Bayes Classifier
  * 조건부 확률과 베이즈 확률에 기반
  * 조건이 많아질수록 계산이 어려움
2. Naive Bayes Classifier
  * 계산을 쉽게 만듦

# Naive Bayes Classifier 실습
### spam mail 분류

# 함수화 시킬 것

In [9]:
from collections import defaultdict
import pandas as pd
import numpy as np
import math

k = 0.5 # 임의의 상수 

input_file = pd.read_csv('9_spam.csv')
# input_file
training_set = np.array(input_file)
print(training_set)

[['me free lottery' 1]
 ['free get free you' 1]
 ['you free scholarship' 0]
 ['free to contact me' 0]
 ['you won award' 0]
 ['you ticket lottery' 1]]


### 토큰 빈도수 및 문서별 토큰 수 계산 (확률 계산을 위한 준비)

In [26]:
# 범주에 속하는 토큰수 세기 1 = (예. 긍정), 0 = (예. 부정)
doccnt1 = 0
doccnt0 = 0

# 토큰 별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0]) # 채워나갈 빈 dictinary 생성
for doc, point in training_set: # training_set을 문자와 숫자로 구분 
    words = doc.split() # 단어를 split을 사용하여 토큰화 
    print(words, point) 
    print('-'*15)
    for word in words: # 만든 단어 token을 하나씩 가져옴
        if point == 1: # 만약 스팸이면,
            wordfreq[word][0] += 1 # 스팸으로 분류할 곳에 1 추가
        else: # 스팸이 아니면 
            wordfreq[word][1] += 1 # 스팸으로 아니라고 분류할 곳에 1 추가 
            
for key, (cnt1, cnt0) in wordfreq.items(): # ???
    if cnt1 > 0: 
        doccnt1 += 1 # 문서수 카운트
    if cnt0 > 0:
        doccnt0 += 1
        
print(wordfreq)
print(doccnt1, doccnt0)

['me', 'free', 'lottery'] 1
---------------
['free', 'get', 'free', 'you'] 1
---------------
['you', 'free', 'scholarship'] 0
---------------
['free', 'to', 'contact', 'me'] 0
---------------
['you', 'won', 'award'] 0
---------------
['you', 'ticket', 'lottery'] 1
---------------
defaultdict(<function <lambda> at 0x0000000009E239D8>, {'me': [1, 1], 'free': [3, 2], 'lottery': [2, 0], 'get': [1, 0], 'you': [2, 2], 'scholarship': [0, 1], 'to': [0, 1], 'contact': [0, 1], 'won': [0, 1], 'award': [0, 1], 'ticket': [1, 0]})
6 8


### Training : 토큰 별 조건부 확률 계산

In [27]:
wordprobs = defaultdict(lambda : [0, 0]) # dict을 만들 lambda 함수 생성 
for key, (cnt1, cnt0) in wordfreq.items(): # 
    wordprobs[key][0] = (cnt1 + k) / (doccnt1 + 2*k)
    wordprobs[key][1] = (cnt0 + k) / (doccnt0 + 2*k)
    
print(wordprobs)

defaultdict(<function <lambda> at 0x000000000A53FAE8>, {'me': [0.21428571428571427, 0.16666666666666666], 'free': [0.5, 0.2777777777777778], 'lottery': [0.35714285714285715, 0.05555555555555555], 'get': [0.21428571428571427, 0.05555555555555555], 'you': [0.35714285714285715, 0.2777777777777778], 'scholarship': [0.07142857142857142, 0.16666666666666666], 'to': [0.07142857142857142, 0.16666666666666666], 'contact': [0.07142857142857142, 0.16666666666666666], 'won': [0.07142857142857142, 0.16666666666666666], 'award': [0.07142857142857142, 0.16666666666666666], 'ticket': [0.21428571428571427, 0.05555555555555555]})


### Classify : 신규 텍스트가 주어졌을 때 확률 계산

In [29]:
doc = 'free lottery'
tokens = doc.split()

# 초기값은 모두 0으로 처리
log_prob1 = log_prob0 = 0.0 # 이렇게 코드를 짜도 된다구?!?! 

# 모든 단어에 대해 반복
for word, (prob1, prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 += math.log(prob1)
        log_prob0 += math.log(prob0)
    else:
        log_prob1 += math.log(1.0 - prob1)
        log_prob0 += math.log(1.0 - prob0)
        
log_prob1 += math.log(doccnt1/len(wordfreq))
log_prob0 += math.log(doccnt0/len(wordfreq))

prob1 = math.exp(log_prob1)
prob0 = math.exp(log_prob0)

print(doc)
print("스팸확률 : {}%".format(prob1 / (prob1 + prob0)*100))
print("정상확률 : {}%".format(prob0 / (prob1 + prob0)*100))

free lottery
스팸확률 : 89.64758762435878%
정상확률 : 10.352412375641222%
